In [2]:
import os
import pandas as pd
from PIL import Image
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
# Paths
image_dir = r"D:\APTOS data\fyp_processed_dataset\combined_images"
csv_path = r"D:\APTOS data\fyp_processed_dataset\cleaned_copy.csv"

In [8]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
import numpy as np

# Dataset Class
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, csv_file, image_dir):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx, 0] + ".png")
        image = Image.open(img_name).convert("RGB")
        label = self.data.iloc[idx, 1]
        # Convert image to tensor, since preprocessing is already done
        image = torch.tensor(
            np.array(image).transpose(2, 0, 1), dtype=torch.float32
        ) / 255.0  # Scale to [0, 1]
        return image, torch.tensor(label, dtype=torch.float32)

# Load dataset
dataset = DiabeticRetinopathyDataset(csv_path, image_dir)

In [10]:
# Split dataset into train and test
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [16]:
# Model Definitions
def get_model(model_name):
    if model_name == "densenet":
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, 1)
    elif model_name == "resnet":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 1)
    return model.to(device)


In [17]:
from tqdm import tqdm

# Training Function with tqdm
def train_model(model, train_loader, num_epochs=10):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch")
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Update progress bar with current loss
            progress_bar.set_postfix(loss=loss.item())

        # Print epoch summary
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")



In [18]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = torch.sigmoid(model(images))
            predictions = (outputs > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {100 * correct / total:.2f}%")

In [19]:
# Main Script
for model_name in ["densenet", "resnet"]:
    print(f"\nTraining {model_name.upper()}...")
    model = get_model(model_name)
    train_model(model, train_loader, num_epochs=1)
    print(f"Evaluating {model_name.upper()}...")
    evaluate_model(model, test_loader)


Training DENSENET...


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:32<00:00, 1.00MB/s]
Epoch 1/1: 100%|██████████| 548/548 [30:54<00:00,  3.38s/batch, loss=0.00742]


Epoch 1/1, Loss: 0.1765
Evaluating DENSENET...
Accuracy: 95.11%

Training RESNET...


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:57<00:00, 1.79MB/s]
Epoch 1/1: 100%|██████████| 548/548 [33:27<00:00,  3.66s/batch, loss=0.117]  


Epoch 1/1, Loss: 0.2264
Evaluating RESNET...
Accuracy: 92.65%
